<a href="https://colab.research.google.com/github/getaccept/notebooks/blob/master/API_copy_products_to_entities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Copy product library using GetAccept *API*

Functions to copy one or many products from a main entity to one or many subentities

In [ ]:
# import dependencies
import requests
import os
import io
import json
import copy
import mimetypes
import ipywidgets as widgets
from google.colab import files
from PIL import Image

#Constants 
BASE_URL = "https://api.getaccept.com/v1"
INT_BASE_URL = "https://int.getaccept.com"
destination_entities = []
SOURCE_ENTITY_ID = ""
last_header = ""
#@title ↓↓ Click here to start
#@markdown This step might take a few seconds to run. <br>
#@markdown Then use __shift+enter__ key or click ► left of each step to go through the flow

In [ ]:
email_widget = widgets.Text(
    value="",
    placeholder="Enter login email",
    description="Email:",
    disabled=False
)
password_widget = widgets.Password(
    value="",
    placeholder="Enter password",
    description="Password:",
    disabled=False
)
#@markdown Use the form below to fill in login details to your entity in GetAccept and then run next cell to login
widgets.VBox([email_widget, password_widget])

In [ ]:
#@markdown Login and store API token
if email_widget.value and password_widget.value:
  payload = { "email": email_widget.value, "password": password_widget.value}
  if SOURCE_ENTITY_ID != "":
    payload["entity_id"] = SOURCE_ENTITY_ID
  response = requests.post(BASE_URL+"/auth", json=payload)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  else:
    raise TypeError(data["errors"], "Please check your credentials")
  # Check login and list entities
  response = requests.get(BASE_URL+"/users/me", headers=source_auth_headers)
  user_data = response.json()
  print("Logged in as " + user_data["user"]["first_name"] + " on entity " + user_data["user"]["entity_name"])
  SOURCE_ENTITY_ID = user_data["user"]["entity_id"]
else:
  raise TypeError("Could not login, missing email or password!")

In [ ]:
#@markdown Select the source entity you would like to get product data from. When you're done, run the next cell
source_entity_list = list(map(lambda x: (x["name"],x["id"]), user_data["entities"]))
source_entity_picker = widgets.Select(
    options=sorted(source_entity_list),
    value=SOURCE_ENTITY_ID,
)
source_entity_picker

In [ ]:
#@markdown Verifying entity token of source...
if source_entity_picker.value != SOURCE_ENTITY_ID:
  # Switch entity
  response = requests.get(BASE_URL+"/refresh/"+source_entity_picker.value, headers=source_auth_headers)
  data = response.json()
  if "access_token" in data:
    source_auth_headers = { "Authorization": "bearer " + data["access_token"]}
  SOURCE_ENTITY_ID = source_entity_picker.value
print("Authenticated to source entity \"%s\"" % source_entity_picker.label)

In [ ]:
#@markdown Select source product(s) to copy:<br>
#@markdown _Hold down __Command__ to select multiple vales_<br>
#@markdown _Click __Command + A__ to select all products_<br>
def get_products(destination_auth_headers):
  payload = {"operationName":"getAllProducts","variables":{"searchTerms":"","limit":100,"offset":0,"sort":"desc","sortBy":"createdAt"},"query":"query getAllProducts($searchTerms: String!, $limit: Int!, $offset: Int!, $sort: String!, $sortBy: SortAttributes!) {\n  getAllProducts(searchTerms: $searchTerms, limit: $limit, offset: $offset, sort: $sort, sortBy: $sortBy) {\n    products {\n      ...productListFragment\n      __typename\n    }\n    totalCount\n    __typename\n  }\n}\n\nfragment productListFragment on Product {\n  id\n  sku\n  name\n  price\n  tax\n  description\n  metaData {\n    locale\n    currency\n    __typename\n  }\n  updatedByFullName\n  updatedAt\n  createdByFullName\n  createdAt\n  __typename\n}\n"}
  result = requests.post(INT_BASE_URL+"/graphql?op=getAllProducts", json=payload, headers=destination_auth_headers)
  product_result = result.json()
  if product_result.get("errors"):
    print(payload)
    print(product_result)
    raise TypeError("Could not get products!")
  else:
    products = product_result["data"]["getAllProducts"]["products"]
    return products

source_products = get_products(source_auth_headers)
product_list = list(map(lambda x: (x["name"],x["id"]), source_products))
source_product_picker = widgets.SelectMultiple(
    options=product_list,
    value=[]
)
source_product_picker

In [ ]:
#@markdown Select destination entities to copy product(s) to:<br>
#@markdown _Hold down __Command__ to select multiple vales_
destination_entity_list = list(map(lambda x: (x["name"],x["id"]), list(filter(lambda x: x["id"] != SOURCE_ENTITY_ID, user_data["entities"]))))
destination_picker = widgets.SelectMultiple(
    options=sorted(destination_entity_list),
    rows=10
)
overwrite_widget = widgets.Checkbox(
    value=False,
    description='Overwrite existing product/SKU',
    disabled=False
)
widgets.VBox([destination_picker, overwrite_widget])


In [ ]:
#@markdown Get auth tokens for destination entities...
if len(destination_picker.value) > 0:
  if last_header == "":
    last_header = source_auth_headers
  destination_entities = []
  for i,entity_id in enumerate(destination_picker.value):
    response = requests.get(BASE_URL+"/refresh/"+entity_id, headers=last_header)
    data = response.json()
    if "access_token" in data:
      last_header = { "Authorization": "bearer " + data["access_token"]}
      destination_entities.append({
        "id": entity_id,
        "name": destination_picker.label[i],
        "header": last_header
      })
      print("Authenticated to destination entity \"%s\"" % destination_picker.label[i])
    else:
      raise TypeError(data["errors"], "Could not authenicate to entity " + destination_picker.label[i])
else:
  raise TypeError("No entities selected")

In [ ]:
#@markdown Preparing product functions
def create_product(prod, destination_auth_headers):
  if prod["metaData"].get("__typename"):
    del prod["metaData"]["__typename"]
  payload = {"operationName":"CreateProduct","variables":{"payload":{"name":prod["name"],"sku":prod["sku"],"description":prod["description"],"price":prod["price"],"tax":prod["tax"],"metaData":prod["metaData"]}},"query":"mutation CreateProduct($payload: CreateProductInput!) {\n  createProduct(payload: $payload)\n}\n"}
  result = requests.post(INT_BASE_URL+"/graphql?op=CreateProduct", json=payload, headers=destination_auth_headers)
  create_product_result = result.json()
  if create_product_result.get("errors"):
    print(payload)
    print(create_product_result)
    raise TypeError("Could not create product!")
  else:
    if create_product_result["data"]["createProduct"] == True:
      print("- Product \"" + prod["name"] + "\" added")
    else:
      print(create_product_result)
      print("Couldn't add product")

def update_product(prod, destination_product_id, destination_auth_headers):
  if prod["metaData"].get("__typename"):
    del prod["metaData"]["__typename"]
  payload = {"operationName":"UpdateProduct","variables":{"id": destination_product_id, "payload":{"name":prod["name"],"sku":prod["sku"],"description":prod["description"],"price":prod["price"],"tax":prod["tax"],"metaData":prod["metaData"]}},"query":"mutation UpdateProduct($id: String!, $payload: UpdateProductInput!) {\n  updateProduct(id: $id, payload: $payload)\n}\n"}
  result = requests.post(INT_BASE_URL+"/graphql?op=UpdateProduct", json=payload, headers=destination_auth_headers)
  update_product_result = result.json()
  if update_product_result.get("errors"):
    print(payload)
    print(update_product_result)
    raise TypeError("Could not update product!")
  else:
    if update_product_result["data"]["updateProduct"] == True:
      print("- Product \"" + prod["name"] + "\" updated")
    else:
      print("Couldn't update product")


def sync_product(source_product_id, destination_products, destination_auth_headers):
  # Sync product between entities
  for source_prod in source_products:
    if source_prod["id"] == source_product_id:
      # Check if product sku exists
      exists = False
      for dest_prod in destination_products:
        if source_prod["sku"] == dest_prod["sku"]:
          exists = True
          break
      if exists and overwrite_widget.value:
        update_product(source_prod, dest_prod["id"], destination_auth_headers)
      if not exists:
        create_product(source_prod, destination_auth_headers)

print("Ready to copy %d product(s) to %d %s" % (len(source_product_picker.value), len(destination_entities), "entity" if len(destination_entities) == 1 else "entities"))

In [ ]:
#@markdown Running copy-functions for each destination entity:
for dest in destination_entities:
  print("Destination entity: \"" + dest["name"] + "\":")
  destination_products = get_products(dest["header"])
  
  for i,source_product_id in enumerate(source_product_picker.value):
    product_name = source_product_picker.label[i]
    sync_product(source_product_id, destination_products, dest["header"])
    
print("All products has been copied")